In [88]:
#Librerias que vamos a usar
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [89]:
data= pd.read_csv('shots.csv')

In [90]:
#Flags usadas
data=data[['PLAYER_ID','PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME','SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA',
       'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG']]

In [91]:
#Calcular los Puntos de cada judagor
data["3pa"]=np.where((data["SHOT_DISTANCE"]>=24), 1, 0)
data['2pa'] = np.where((data["SHOT_DISTANCE"]<24), 1, 0)
data["3pm"]=np.where((data["SHOT_MADE_FLAG"]==1) & (data["SHOT_DISTANCE"]>=24), 3, 0)
data['2pm'] = np.where((data["SHOT_MADE_FLAG"]==1) & (data["SHOT_DISTANCE"]<24), 2, 0)
data['Pts']=data["3pm"]+data['2pm']

In [92]:
#Flags usadas para crear las dummies
data=data[["Pts","SHOT_ZONE_BASIC","PLAYER_NAME","PLAYER_ID",'TEAM_NAME']]

In [93]:
#Crear Dummies para el modelo
data["Mid FGA"]=np.where(data["SHOT_ZONE_BASIC"]=='Mid-Range',1,0)
data["Restricted Area FGA"]=np.where(data["SHOT_ZONE_BASIC"]=='Restricted Area',1,0)
data["Above the Break 3 FGA"]=np.where(data["SHOT_ZONE_BASIC"]=='Above the Break 3',1,0)
data["Right Corner 3 FGA"]=np.where(data["SHOT_ZONE_BASIC"]=='Right Corner 3',1,0)
data['In The Paint (Non-RA) FGA']=np.where(data["SHOT_ZONE_BASIC"]=='In The Paint (Non-RA)',1,0)
data["Left Corner 3 FGA"]=np.where(data["SHOT_ZONE_BASIC"]=='Left Corner 3',1,0)

In [95]:
#Ordenar los datos para poder hacer el merge despues
data=data.sort_values("PLAYER_ID")

In [114]:
#Modelo general
x=data[['Mid FGA', 'Restricted Area FGA',
       'Above the Break 3 FGA', 'Right Corner 3 FGA', 'In The Paint (Non-RA) FGA',
       'Left Corner 3 FGA']]
y=data["Pts"]
model_l = sm.OLS(y, x).fit()
print(model_l.summary())
betas = model_l.params
pred_pps=model_l.predict(x).values

                                 OLS Regression Results                                
Dep. Variable:                    Pts   R-squared (uncentered):                   0.461
Model:                            OLS   Adj. R-squared (uncentered):              0.461
Method:                 Least Squares   F-statistic:                          7.083e+04
Date:                Mon, 22 Mar 2021   Prob (F-statistic):                        0.00
Time:                        17:25:58   Log-Likelihood:                     -7.6429e+05
No. Observations:              496182   AIC:                                  1.529e+06
Df Residuals:                  496176   BIC:                                  1.529e+06
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                                coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [97]:
#Modelo con las betas de cada jugador, convertirlo en una Base de Datos para utilizarlo por jugador
df=pd.DataFrame()
for i in data["PLAYER_NAME"].unique():
    

    data_player=data.loc[data["PLAYER_NAME"]=='{}'.format(i)]
    x_data_player=data_player[['Mid FGA', 'Restricted Area FGA',
           'Above the Break 3 FGA', 'Right Corner 3 FGA', 'In The Paint (Non-RA) FGA',
           'Left Corner 3 FGA']]
    y_data_player=data_player["Pts"]
    model_jugador = sm.OLS(y_data_player, x_data_player).fit()
    betas_jugador = model_jugador.params
    df=df.append(betas_jugador,ignore_index=True)

In [99]:
att=data.groupby("PLAYER_ID").sum()

In [100]:
#Calcular los tiros por jugador
att["FGA"]=att["Mid FGA"]+att["Restricted Area FGA"]+att["Above the Break 3 FGA"]+att["Right Corner 3 FGA"]+att["In The Paint (Non-RA) FGA"]+att["Left Corner 3 FGA"]

In [102]:
player_id=data["PLAYER_ID"].unique()

In [105]:
#Utilizar como index el Player Id para el merge
df=df.set_index(player_id)

In [106]:
#Merge para la Base final
df=df.merge(att,left_index=True,right_index=True)

In [110]:
#Agregamos la columna con los nombres
df["Name"]=data["PLAYER_NAME"].unique()

In [111]:
#Las nombramos para evitar confusión
df.columns=['Above the Break 3', 'In The Paint (Non-RA)', 'Left Corner 3', 'Mid','Restricted Area', 'Right Corner 3','FGA','Above the Break 3 FGA', 'In The Paint (Non-RA) FGA','Left Corner 3 FGA', 'Mid FGA', 'Restricted Area FGA','Right Corner 3 FGA',"Pts","Name"]

In [112]:
#Dar el nuevo valor a las columnas
cols = df.columns.tolist()
#Hacer que el Nombre esté al principio
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [113]:
#Así queda la base
df.head()

,Name,Above the Break 3,In The Paint (Non-RA),Left Corner 3,Mid,Restricted Area,Right Corner 3,FGA,Above the Break 3 FGA,In The Paint (Non-RA) FGA,Left Corner 3 FGA,Mid FGA,Restricted Area FGA,Right Corner 3 FGA,Pts
1713,Vince Carter,1.038186,0.906250,8.604651e-01,0.772727,1.106796,0.826923,755,88,103,419,52,64,43,769
1717,Dirk Nowitzki,0.934426,0.962963,4.000000e-01,0.791367,1.200000,0.750000,327,139,5,183,16,27,5,375
2037,Jamal Crawford,1.072727,0.957746,3.636364e-01,0.868966,1.000000,0.733333,412,145,22,165,15,71,22,440
2199,Tyson Chandler,0.000000,0.857143,0.000000e+00,0.000000,1.333333,0.000000,150,1,108,1,0,7,0,117
2200,Pau Gasol,1.800000,0.750000,8.593432e-16,0.761905,1.055556,0.000000,90,21,36,10,1,24,2,94
